## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Albany,42.6001,-73.9662,21.31,77,10,4.18,US,clear sky
1,1,Areosa,41.7178,-8.8571,50.32,68,57,5.66,PT,broken clouds
2,2,Avarua,-21.2078,-159.7750,80.65,78,20,9.22,CK,few clouds
3,3,Imamoglu,37.2651,35.6572,35.31,56,0,3.78,TR,clear sky
4,4,Ushuaia,-54.8000,-68.3000,60.46,67,40,14.97,AR,scattered clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Enter your desired minimum temperature for your vacation."))
max_temp = float(input("Enter your desired maximum temperature for your vacation."))

Enter your desired minimum temperature for your vacation.75
Enter your desired maximum temperature for your vacation.100


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & 
                                     (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
2,2,Avarua,-21.2078,-159.7750,80.65,78,20,9.22,CK,few clouds
10,10,Namatanai,-3.6667,152.4333,85.15,69,100,7.52,PG,overcast clouds
14,14,Iranduba,-3.2847,-60.1861,77.32,94,100,2.39,BR,overcast clouds
16,16,Minbu,20.1833,94.8833,75.81,26,90,5.64,MM,overcast clouds
17,17,Maragogi,-9.0122,-35.2225,79.30,72,61,4.72,BR,broken clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Current Description    0
dtype: int64

In [48]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [49]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,80.65,few clouds,-21.2078,-159.7750,
10,Namatanai,PG,85.15,overcast clouds,-3.6667,152.4333,
14,Iranduba,BR,77.32,overcast clouds,-3.2847,-60.1861,
16,Minbu,MM,75.81,overcast clouds,20.1833,94.8833,
17,Maragogi,BR,79.30,broken clouds,-9.0122,-35.2225,
21,Hobyo,SO,75.52,overcast clouds,5.3505,48.5268,
32,Esperance,AU,76.84,few clouds,-33.8667,121.9000,
33,Touros,BR,79.70,few clouds,-5.1989,-35.4608,
34,Kapaa,US,78.78,few clouds,22.0752,-159.3190,
36,Hobart,AU,81.01,few clouds,-42.8794,147.3294,


In [50]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [51]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df_cleaned = hotel_df.loc[(hotel_df["Hotel Name"] != "")]
hotel_df_cleaned.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,80.65,few clouds,-21.2078,-159.7750,Paradise Inn
10,Namatanai,PG,85.15,overcast clouds,-3.6667,152.4333,Seagulls Inn Namatani
14,Iranduba,BR,77.32,overcast clouds,-3.2847,-60.1861,Evolução Ecolodge
16,Minbu,MM,75.81,overcast clouds,20.1833,94.8833,Pan Tha Tin Guest House - Burmese Only
17,Maragogi,BR,79.30,broken clouds,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort


In [52]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df_cleaned.to_csv(output_data_file, index_label = "City_ID")

In [53]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_cleaned.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df_cleaned[["Lat", "Lng"]]

In [54]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))